In [1]:
import os, sys, time
from copy import deepcopy

import utils.helpers.constants as path
from utils.helpers.data import * 
from utils.helpers.plot import * 
from utils.helpers.video import * 

import cv2
import numpy as np
from tensorflow.keras import models

In [3]:
final_model = models.load_model('models_prof/vgg16_a__9.h5')

In [4]:
__learning_rate__ = 1e-5
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='accuracy', patience=5)
lr_reduce = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5, verbose=1, mode='max', min_lr=__learning_rate__)
filepath="models/weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor= 'accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [5]:
mean_ious = []
for video_filename in os.listdir(path.__video_path__) : 

    video_path = os.path.join(path.__video_path__, video_filename)

    list_bbox_filename = video_filename.split('.')[0] +"_2_bboxes.txt"
    list_bbox_path = os.path.join(path.__bboxes_path__,list_bbox_filename)

    num_first_frame_with_coord  = get_num_first_frame_with_coord(list_bbox_path)

    debut_video = time.time()
    print(f"Traitement de la vidéo : {video_filename}")


    __batch_size__ = 32
    class_labels, nb_labels = get_labels()

    pred_boxes = []
    num_first_frame, coords_first_box = get_num_first_frame_with_coord(list_bbox_path)
    pred_boxes.append(coords_first_box)

    true_boxes = get_all_true_box(list_bbox_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("ERROR: unable to read video:", video_path)
        sys.exit()

    label = video_filename.split("Place")[0]
    label_id = class_labels.index(label)


    new_imgs_for_model = []
    new_labels_for_model = []

    i = 0
    sum_ious = 0
    nb_frame_with_box = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            break

        if i > num_first_frame : 
            print(f"Frame n°{i}")
            new_boxes = generate_rdm_boxes_from_box(
                pred_boxes[-1],
                0.4,
                0.1,
                frame.shape[1],
                frame.shape[0],
                nb_box=__batch_size__,
            )

            batch_imgs = []
            img_to_show = deepcopy(frame)
            for j, box in enumerate(new_boxes):
                img = frame[box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
                batch_imgs.append(cv2.resize(img, dsize=(227, 227)))

            batch_imgs = np.array(batch_imgs)
            model_prediction_batch = final_model.predict(batch_imgs)

            proba = [[pr[i] for pr in model_prediction_batch] for i in range(nb_labels)]

            best_box = new_boxes[np.argmax(proba[label_id])]
            if(true_boxes[i]) : 
                iou = compute_IOU(best_box, true_boxes[i])
                print(f'iou : {iou}')
                sum_ious += iou 
                nb_frame_with_box += 1
                
                img_best_box = frame[best_box[1] : best_box[1] + best_box[3], best_box[0] : best_box[0] + best_box[2]]
                img_best_box_resized = preprocess_image(img_best_box)

                new_imgs_for_model.append(img_best_box_resized)
                new_img_label = np.zeros(5)
                if(iou > 0.5) : 
                    new_img_label[label_id] = 1
                new_labels_for_model.append(new_img_label)
            pred_boxes.append(best_box)

        i += 1


    print(f"Mean IOU = {sum_ious/nb_frame_with_box}")
    mean_ious.append(sum_ious/nb_frame_with_box)

    for layer in final_model.layers[:-2] : 
        layer.trainable = False

    opt = Adam(learning_rate=__learning_rate__)
    final_model.compile(loss="categorical_crossentropy", optimizer = opt, metrics=["accuracy"])
    new_x = np.asarray(new_imgs_for_model)
    new_y = np.asarray(new_labels_for_model)
    history = final_model.fit(new_x, new_y, epochs=20,callbacks=[lr_reduce, early_stopping, checkpoint] )


    fin_video = time.time()
    print(f"La vidéo a été traité en {fin_video - debut_video}")


Traitement de la vidéo : BowlPlace1Subject1.mp4
Frame n°96
iou : 0.5220899902184545
Frame n°97
iou : 0.39418293149636435
Frame n°98
iou : 0.20745309014565017
Frame n°99
iou : 0.7337740384615384
Frame n°100
iou : 0.616545775338169
Frame n°101
iou : 0.5709288767975126
Frame n°102
iou : 0.18128409567771717
Frame n°103
iou : 0.23929811029695333
Frame n°104
iou : 0.20834708795095497
Frame n°105
iou : 0.07817674742896985
Frame n°106
iou : 0.20018055501040155
Frame n°107
iou : 0.2523110847256382
Frame n°108
iou : 0.34995922805110086
Frame n°109
iou : 0.41134868421052634
Frame n°110
iou : 0.4222861842105263
Frame n°111
iou : 0.38318306546452374
Frame n°112
iou : 0.4243374627567822
Frame n°113
iou : 0.10884594447468698
Frame n°114
iou : 0
Frame n°115
iou : 0
Frame n°116
iou : 0
Frame n°117
iou : 0
Frame n°118
iou : 0
Frame n°119
iou : 0
Frame n°120
iou : 0
Frame n°121
iou : 0
Frame n°122
iou : 0
Frame n°123
iou : 0
Frame n°124
iou : 0
Frame n°125
iou : 0
Frame n°126
iou : 0
Frame n°127
iou : 0
